In [1]:
from pprint import pprint
from collections import defaultdict
import json
import re
import sys
from typing import Union, List, Optional, Set, Tuple, Dict, Optional, Callable

import numpy as np
from lab.utils import shorten
import pandas as pd
from pathlib import Path
from machine_learning.analysis.dataframe import (
    pivot_rotate,

    slice_rows,
    slice_cols,
    sort_rows,
    sort_cols,
    aggregate,
    percentize,
    round,

    rename_index,
    rename_cols,
    rename_cells,

    isnan,
    to_latex,
    color_by_rank,
)
from machine_learning.analysis.series import (
    maybe_numeric_series,
)
from machine_learning.analysis.utils import (
    maybe_round,
)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 1000)

/home/acb11878tj/.pyenv/versions/3.8.5/envs/jupyter-analysis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load functions

In [2]:
def find_results(top_dir: Union[str, Path], regexps: Optional[List[str]] = None) -> None:
    regexps = regexps or []
    top_dir = Path(top_dir)
    for path in sorted([
        path for path in top_dir.glob('**/results.tsv')
        if all(re.match(regexp, str(path)) for regexp in regexps)
    ]):
        print(path)

In [3]:
# COL_METHOD = 'checkpoint_name'
COL_METHOD = 'method'
COL_MODEL_NAME_OR_PATH = 'model_name_or_path'
COL_DATASET = 'dataset_uname'
COL_TASK = 'dataset_uname'

# Analysis functions

In [4]:
def name_method(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
            
    df[COL_METHOD] = df.apply(
        lambda row: row[COL_MODEL_NAME_OR_PATH],
        axis=1,
    )
    
    return df


def prettify_df(df: pd.DataFrame) -> pd.DataFrame:
    df = percentize(df)
    df = round(df)
    return df

In [5]:
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230826.jpn/'
_TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230901.overfit/'
find_results(_TOP_DIR)

../outputs/02.aggregate_tf_results.py/20230901.overfit/results.tsv


In [6]:
# results_path = '../outputs/02.aggregate_tf_results.py/20230826.jpn/results.tsv'
results_path = '../outputs/02.aggregate_tf_results.py/20230901.overfit/results.tsv'


In [7]:
master_df = pd.read_csv(results_path, sep='\t')
master_df

,base_config_name,dataset_uname,generation_max_proof_steps,generation_num_beams,generation_input_k,gradient_accumulation_steps,learning_rate,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,model_name_or_path,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,seed,shot,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,eval/extr_stps.D-all.proof_accuracy.zero_one,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy
0,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,64,0.0001,seq2seq,False,0.5,1,30,1000,300,1000,1,google/mt5-large,1,1,all_at_once,True,0,NaN,Solve FLD task:,max_length,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,16,0.0001,seq2seq,False,0.5,1,30,1000,300,1000,1,retrieva-jp/t5-large-long,4,4,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,8,0.0001,seq2seq,False,0.5,1,30,1000,300,100,1,retrieva-jp/t5-base-long,8,8,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,8,0.0001,seq2seq,False,0.5,1,30,1000,300,100,1,retrieva-jp/t5-small-long,8,8,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,8,0.0001,seq2seq,False,0.5,1,30,1000,300,100,1,megagonlabs/t5-base-japanese-web,8,8,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,FLNLcorpus.20220827.base,20230901.random_t

In [8]:
method_named_df = name_method(master_df)
method_named_df

,base_config_name,dataset_uname,generation_max_proof_steps,generation_num_beams,generation_input_k,gradient_accumulation_steps,learning_rate,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,model_name_or_path,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,seed,shot,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,eval/extr_stps.D-all.proof_accuracy.zero_one,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy,method
0,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,64,0.0001,seq2seq,False,0.5,1,30,1000,300,1000,1,google/mt5-large,1,1,all_at_once,True,0,NaN,Solve FLD task:,max_length,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,google/mt5-large
1,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,16,0.0001,seq2seq,False,0.5,1,30,1000,300,1000,1,retrieva-jp/t5-large-long,4,4,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,retrieva-jp/t5-large-long
2,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,8,0.0001,seq2seq,False,0.5,1,30,1000,300,100,1,retrieva-jp/t5-base-long,8,8,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,retrieva-jp/t5-base-long
3,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,8,0.0001,seq2seq,False,0.5,1,30,1000,300,100,1,retrieva-jp/t5-small-long,8,8,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,retrieva-jp/t5-small-long
4,FLNLcorpus.20220827.base,20230901.random_transitive_verbs.D3,20,2,NaN,8,0.0001,seq2seq,False,0.5,1,30,1000,300,100,1,megagonlabs/t5-base-japanese-web,8,8,all_at_once,True,0,NaN,Solve FLD task:,longest,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN

In [9]:
TASK_NAMES = [
    '20230826.jpn.D3',
    '20230826.jpn.D8',
]
MAJOR_TASK = TASK_NAMES[0]

task_dfs: Dict[str, pd.DataFrame] = {}
for task_name in TASK_NAMES:
    task_dfs[task_name] = slice_rows(
        method_named_df,
        lambda row: row[COL_TASK] == task_name
    )

task_dfs[MAJOR_TASK]

,base_config_name,dataset_uname,generation_max_proof_steps,generation_num_beams,generation_input_k,gradient_accumulation_steps,learning_rate,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,model_name_or_path,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,seed,shot,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,eval/extr_stps.D-all.proof_accuracy.zero_one,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy,method


In [10]:
METRIC_NAMES = [
    'eval/extr_stps.D-all.proof_accuracy.zero_one',
    'eval/strct.D-all.proof_accuracy.zero_one',
    'eval/strct.D-all.answer_accuracy',
]

metric_dfs: Dict[str, pd.DataFrame] = {}
for task_name, task_df in task_dfs.items():
    metric_dfs[task_name] = slice_cols(task_df, [COL_METHOD] + METRIC_NAMES)

metric_dfs[MAJOR_TASK]

,method,eval/extr_stps.D-all.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/strct.D-all.answer_accuracy


In [11]:
METRIC_RENAMES = {
    'eval/extr_stps.D-all.proof_accuracy.zero_one': 'prf_acc.extr',
    'eval/strct.D-all.proof_accuracy.zero_one': 'prf_acc.strct',
    'eval/strct.D-all.answer_accuracy': 'ans.acc',
}

pretty_dfs: Dict[str, pd.DataFrame] = {}

for task_name, metric_df in metric_dfs.items():
    pretty_df = prettify_df(metric_df)
    pretty_df = rename_cols(pretty_df, METRIC_RENAMES)
    
    pretty_df.index = pretty_df[COL_METHOD]
    pretty_df = pretty_df.drop(columns=[COL_METHOD])
    
    pretty_df = color_by_rank(pretty_df, 'col')
    
    pretty_dfs[task_name] = pretty_df

pretty_dfs[MAJOR_TASK]

,prf_acc.extr,prf_acc.strct,ans.acc
method,,,


In [12]:
def horizontal_concat(dfs: List[pd.DataFrame]) -> pd.DataFrame:
    dfs = [df.copy() for df in dfs]
    # align index for horizontal concat
    for df in dfs:
        df.index = range(len(df))
    return pd.concat(dfs, axis=1)
    
colored_concat_df = horizontal_concat(
    [pretty_df for task_name, pretty_df in sorted(pretty_dfs.items())]
)
colored_concat_df

,prf_acc.extr,prf_acc.strct,ans.acc,prf_acc.extr,prf_acc.strct,ans.acc


In [13]:
print(to_latex(colored_concat_df, with_index=True))

\begin{tabular}{lrrrrrr}
\toprule
 & prf\_acc.extr & prf\_acc.strct & ans.acc & prf\_acc.extr & prf\_acc.strct & ans.acc \\
\midrule
\bottomrule
\end{tabular}

